# Creating own Chatbot

<img src="CC.png" width=400/>

## Introduction

In my CIS 3990: Introduction to Artificial Intelligence class, we learned how to create our very own chatbots. One of the projects I worked on involved building a chatbot that can convert currencies in real-time!

I implemented the chatbot using the OpenAI API and a real-time exchange rate API. The idea was to let users input an amount in one currency and get the equivalent value in another currency, based on the most up-to-date exchange rates.

It was a fun way to learn how to integrate APIs and apply them in practical use cases like currency conversion. Let's walk through the process and take a look at how the bot works!

---

## The Currency Conversion Bot

### Setting Up the Chatbot

To start, I set up the OpenAI API to handle the conversation. The chatbot responds to user inputs by converting currencies and then returning the result as a structured JSON object.

Here's a breakdown of the code I used to initialize the chatbot:

--------------------------------------------------------------
```
import requests <br>
import json <br>
from openai.types.chat.chat_completion import ChatCompletionMessage <br>


def convert_currency(amount, from_currency, to_currency): <br>
    api_url = f"https://api.exchangerate-api.com/v4/latest/{from_currency.upper()}" <br>
    response = requests.get(api_url) <br>
    data = response.json() <br>

if "rates" not in data or to_currency.upper() not in data["rates"]: <br>
    return "Invalid currency code. Please try again." <br>

exchange_rate = data["rates"][to_currency.upper()] <br>
converted_amount = amount * exchange_rate <br>

return { <br>
        "converted_amount": converted_amount, <br>
        "exchange_rate": exchange_rate <br>
    } <br>
<br>
tools = [{ <br>
    "type": "function", <br>
    "function": { <br>
        "name": "convert_currency", <br>
        "description": "Converts an amount from one currency to another based on real-time exchange rates.", <br>
        "parameters": { <br>
            "type": "object", <br> 
            "properties": { <br>
                "amount": {"type": "number"}, <br>
                "from_currency": {"type": "string"}, <br>
                "to_currency": {"type": "string"} <br>
            }, <br>
            "required": ["amount", "from_currency", "to_currency"], <br>
            "additionalProperties": False <br>
        }, <br>
        "strict": True <br>
    } <br>
}] <br>

--------------------------------------------------------------
### Initialize OpenAI API caller
```
openai_caller = OpenAI_LLM_Caller( <br>
    model="gpt-4o-mini", <br>
    api_key=openai_api_key, <br>
    tools=tools <br>
) <br>

user_input = ""<br> 
system_message = """You are a currency conversion assistant. Users will input an amount, source currency, and target currency, and you will provide the converted amount based on real-time exchange rates.\n\nFormat your response as a JSON:\n{\n  \"message\": \"Your converted amount is X\",\n  \"emoji string\": \"💱🌍\"\n}\n\nExample:\n{\n  \"message\": \"100 USD is equivalent to 85 EUR.\",\n  \"emoji string\": \"💵➡️💶\"\n}\n"""
messages = [{"role": "system", "content": system_message}] <br>

print("Chatbot initialized, type in \"exit\" to end the conversation\n")

while user_input.lower() != "exit": <br>
    assistant_response = openai_caller.generate_completion( <br>
        messages=messages, <br>
        response_format={"type": "json_object"} <br>
    ) <br>

if isinstance(assistant_response, ChatCompletionMessage): <br> 
        messages.append(assistant_response) <br> 
        print("---") <br> 
        openai_caller.print_wrap_text(f"assistant tool call: {assistant_response}") <br> 
<br> 
for tool_call in assistant_response.tool_calls: <br> 
         if tool_call.function.name == "convert_currency": <br> 
            args = json.loads(tool_call.function.arguments) <br> 
            result = convert_currency(args["amount"], args["from_currency"], args["to_currency"]) <br> 
            messages.append({ <br> 
                "role": "tool",<br> 
                "tool_call_id": tool_call.id,　<br> 
                "content": json.dumps(result)　<br> 
            })　<br> 
            openai_caller.print_wrap_text(f"tool call response: {result}")　<br> 

print("---\n")　<br> 
    assistant_response = openai_caller.generate_completion(　<br> 
        messages=messages, <br> 
        response_format={"type": "json_object"} <br> 
        )

 openai_caller.print_wrap_text(f"assistant: {assistant_response}") <br> 
print("\n") <br> 
user_input = input("user: ") <br> 
print("\n") <br> 
   messages.append({"role": "assistant", "content": assistant_response}) <br> 
 messages.append({"role": "user", "content": user_input}) <br> 

print(f"\n----\nTotal Conversation Cost: {openai_caller.total_cost}") <br> 


## How the Conversion Works
When the user types in their input, the chatbot generates a response based on the exchange rate data from the API. The bot fetches the exchange rate for the specified currencies and provides the converted amount.

For example, if the user enters something like "20 USD to CAD", the bot would fetch the real-time exchange rate for USD to CAD and return the converted amount.

Here’s how the chatbot responds when the user inputs currency conversion data:


--------------------------------------------------------------
### Response  <br> 
Chatbot initialized, type in "exit" to end the conversation  <br> 

assistant: {"message":"Please provide the amount, source currency, and target currency for  <br> 
conversion.","emoji string":"💱🌍"}  <br> 


user: 20 us dollars canandian dollars <br> 


 <br> 
assistant tool call: ChatCompletionMessage(content=None, refusal=None, role='assistant',  <br> 
audio=None, function_call=None,  <br> 
tool_calls=[ChatCompletionMessageToolCall(id='call_IiRg1xR6QPAqfUB51HWGpael',  <br> 
function=Function(arguments='{"amount":20,"from_currency":"USD","to_currency":"CAD"}',  <br> 
name='convert_currency'), type='function')])  <br> 
tool call response: {'converted_amount': 28.599999999999998, 'exchange_rate': 1.43}  <br> 
---

assistant: { "message": "20 USD is equivalent to 28.60 CAD.", "emoji string": "💵➡️🇨🇦" }  <br> 


user: exit  <br> 



----  <br> 
Total Conversation Cost: 0.0001392  <br> 


In this example, the bot successfully converted 20 USD to 28.60 CAD using the real-time exchange rate of 1.43. The conversation ends with the "exit" command.



## Takeaway: Building a Useful and Fun Chatbot

Creating a currency conversion chatbot was a great hands-on project to help me understand how AI can be used in real-world applications. It allowed me to experiment with integrating external data (like exchange rates) into AI-generated conversations, and it was fun to see how AI could handle specific tasks based on user input.

### Here's what I learned from the process:

- API integration is key: This project showed me how powerful APIs are when it comes to extending the functionality of chatbots. Fetching real-time data and using it to respond to user queries was a pretty cool feature.

- Fine-tuning conversation flow: I also learned how to set up system messages to guide the chatbot’s responses. This is a great way to ensure that the bot stays focused on the task at hand (in this case, currency conversion).

- The power of JSON formatting: Presenting the chatbot’s responses in JSON format made it easy to structure the conversation, especially when dealing with multiple data points like conversion amounts and currency symbols.

---

## Final Thoughts

This project was a fantastic way to get hands-on with AI and APIs, and it gave me a deeper appreciation for how chatbots can be used to automate tasks and provide value to users. Whether it’s converting currencies or answering customer service questions, chatbots have endless possibilities for making interactions more efficient.

So, if you’re looking to build your own AI chatbot, I highly recommend experimenting with different APIs to add real-time functionality. It’s a rewarding experience that’ll help you get one step closer to mastering conversational AI!
